Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from time import sleep
from os import environ
from datetime import datetime
from six.moves import cPickle as pickle

environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"] = '2'

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
# Logistic
num_labels = 10
pixelCount = 28 * 28
train_subset = 10000
num_steps = 801

graph = tf.Graph()
with graph.as_default():
    # Configure Constants
    tf_TrainSet = tf.constant(train_dataset[:train_subset, :])
    tf_TrainLabels = tf.constant(train_labels[:train_subset, :])
    tf_TestSet = tf.constant(test_dataset)
    tf_ValidSet = tf.constant(valid_dataset)
    
    # Setup the logistic regression variables
    weights = tf.Variable(
        tf.truncated_normal([pixelCount, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    log = tf.matmul(tf_TrainSet, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(log, tf_TrainLabels))
    # L2 regularization for the fully connected parameters.
    regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
    # Add the regularization term to the loss.
    #c = 5e-4
    c = 0.0075
    ### 0.05 - 87.9%
    ### 0.0075 - 88.7%
    ### 0.005 - 88.4%
    ### 0.001 - 84.2%
    ### 0.0005 - 83.2%
    ### 0.0001 - 82.8%
    
    loss += c * regularizers
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_prediction = tf.nn.softmax(log)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_ValidSet, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_TestSet, weights) + biases)
    
with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tm = datetime.now()
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
            print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
            print('Time elapsed:{}'.format((datetime.now()-tm).total_seconds()))

Initialized
Loss at step 0: 38.727264
Training accuracy: 10.4%
Validation accuracy: 12.8%
Test accuracy: 13.2%
Time elapsed:0.556523
Loss at step 100: 11.473408
Training accuracy: 74.0%
Validation accuracy: 71.9%
Test accuracy: 78.9%
Time elapsed:7.879964
Loss at step 200: 5.409733
Training accuracy: 78.2%
Validation accuracy: 76.0%
Test accuracy: 82.8%
Time elapsed:15.443403
Loss at step 300: 2.749806
Training accuracy: 81.3%
Validation accuracy: 78.5%
Test accuracy: 85.6%
Time elapsed:22.410947
Loss at step 400: 1.582616
Training accuracy: 83.3%
Validation accuracy: 80.3%
Test accuracy: 87.1%
Time elapsed:29.446878
Loss at step 500: 1.067767
Training accuracy: 84.3%
Validation accuracy: 81.2%
Test accuracy: 88.0%
Time elapsed:36.429699
Loss at step 600: 0.838353
Training accuracy: 84.8%
Validation accuracy: 81.8%
Test accuracy: 88.3%
Time elapsed:43.481676
Loss at step 700: 0.735221
Training accuracy: 85.0%
Validation accuracy: 82.0%
Test accuracy: 88.6%
Time elapsed:50.524413
Loss a

In [6]:
# Logistic
num_labels = 10
pixelCount = 28 * 28
train_subset = 10000
num_steps = 801

graph = tf.Graph()
with graph.as_default():
    # Configure Constants
    tf_TrainSet = tf.constant(train_dataset[:train_subset, :])
    tf_TrainLabels = tf.constant(train_labels[:train_subset, :])
    tf_TestSet = tf.constant(test_dataset)
    tf_ValidSet = tf.constant(valid_dataset)
    
    # Setup the logistic regression variables
    weights = tf.Variable(
        tf.truncated_normal([pixelCount, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    log = tf.matmul(tf_TrainSet, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(log, tf_TrainLabels))
    # L2 regularization for the fully connected parameters.
    regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
    # Add the regularization term to the loss.
    #c = 5e-4
    c = 0.0075
    ### 0.05 - 87.9%
    ### 0.0075 - 88.7%
    ### 0.005 - 88.4%
    ### 0.001 - 84.2%
    ### 0.0005 - 83.2%
    ### 0.0001 - 82.8%
    
    loss += c * regularizers
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_prediction = tf.nn.softmax(log)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_ValidSet, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_TestSet, weights) + biases)
    
with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tm = datetime.now()
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
            print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
            print('Time elapsed:{}'.format((datetime.now()-tm).total_seconds()))

Initialized
Loss at step 0: 37.877121
Training accuracy: 12.5%
Validation accuracy: 15.9%
Test accuracy: 16.7%
Time elapsed:0.801606
Loss at step 100: 11.684312
Training accuracy: 73.5%
Validation accuracy: 71.6%
Test accuracy: 79.0%
Time elapsed:2.560694
Loss at step 200: 5.506093
Training accuracy: 77.7%
Validation accuracy: 75.5%
Test accuracy: 82.9%
Time elapsed:4.2878
Loss at step 300: 2.789194
Training accuracy: 80.8%
Validation accuracy: 78.1%
Test accuracy: 85.3%
Time elapsed:6.014395
Loss at step 400: 1.598399
Training accuracy: 82.9%
Validation accuracy: 80.1%
Test accuracy: 87.3%
Time elapsed:7.745809
Loss at step 500: 1.073883
Training accuracy: 84.2%
Validation accuracy: 81.0%
Test accuracy: 88.0%
Time elapsed:9.482229
Loss at step 600: 0.840688
Training accuracy: 84.6%
Validation accuracy: 81.7%
Test accuracy: 88.5%
Time elapsed:11.27401
Loss at step 700: 0.736113
Training accuracy: 84.8%
Validation accuracy: 82.0%
Test accuracy: 88.7%
Time elapsed:13.207822
Loss at step 

In [10]:
from datetime import datetime

# Neural implementation
def testNN(c):
    t = datetime.now()
    num_labels = 10
    pixelCount = 28 * 28
    train_subset = 10000
    num_steps = 1001
    hiddenNodes = 1024
    useGPU = 1
    
    graph = tf.Graph()
    with graph.as_default():
        with tf.device('/gpu:1'):
            # Configure Constants
            tf_TrainSet = tf.constant(train_dataset[:train_subset, :])
            tf_TrainLabels = tf.constant(train_labels[:train_subset, :])
            tf_TestSet = tf.constant(test_dataset)
            tf_ValidSet = tf.constant(valid_dataset)

            # Setup the logistic regression variables
            w1 = tf.Variable(tf.truncated_normal([pixelCount, hiddenNodes]))
            b1 = tf.Variable(tf.zeros([hiddenNodes]))
            w2 = tf.Variable(tf.truncated_normal([hiddenNodes, num_labels]))
            b2 = tf.Variable(tf.zeros([num_labels]))


            logitLayer1 = tf.matmul(tf_TrainSet, w1) + b1
            reluLayer1 = tf.nn.relu(logitLayer1)
            logitLayer2 = tf.matmul(reluLayer1, w2) + b2
            loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logitLayer2, tf_TrainLabels))
            # L2 regularization for the fully connected parameters.
            regularizers = (tf.nn.l2_loss(w1) + tf.nn.l2_loss(b1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(b2))
            # Add the regularization term to the loss.
            loss += c * regularizers

            optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

            train_prediction = tf.nn.softmax(logitLayer2)
            valid_prediction = tf.nn.softmax(
                tf.matmul(tf.nn.relu(tf.matmul(tf_ValidSet, w1) + b1), w2) + b2)
            test_prediction = tf.nn.softmax(
                tf.matmul(tf.nn.relu(tf.matmul(tf_TestSet, w1) + b1), w2) + b2)

    with tf.Session(graph=graph) as session:
        # This is a one-time operation which ensures the parameters get initialized as
        # we described in the graph: random weights for the matrix, zeros for the
        # biases. 
        tf.initialize_all_variables().run()
        print('Initialized for c={}'.format(c))
        for step in range(num_steps):
            # Run the computations. We tell .run() that we want to run the optimizer,
            # and get the loss value and the training predictions returned as numpy
            # arrays.
            _, l, predictions = session.run([optimizer, loss, train_prediction])
            if (step % 200 == 0):
                nt = (datetime.now() - t).total_seconds()
                ac = accuracy(test_prediction.eval(), test_labels)
                print("Progress at {} steps after {} seconds has test accuracy of {:.2%}%".format(step, nt, ac/100))
                sleep(10) # Cool down so my laptop GPU doesn't catch fire
            if (step % 1000 == 0 and step > 0):
                print('Loss at step %d: %f' % (step, l))
                print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
                print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
                print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
        print('')
        
vals = [0.0075, 0.005, 0.0005]
for v in vals:
    testNN(v)
    sleep(60) # Cool down so my laptop GPU doesn't catch fire

### At 0.05 - 88%

InvalidArgumentError: Cannot assign a device to node 'Variable_3': Could not satisfy explicit device specification '/device:GPU:1' because no devices matching that specification are registered in this process; available devices: /job:localhost/replica:0/task:0/cpu:0
	 [[Node: Variable_3 = Variable[container="", dtype=DT_FLOAT, shape=[10], shared_name="", _device="/device:GPU:1"]()]]
Caused by op u'Variable_3', defined at:
  File "/home/mike/anaconda2/envs/tfgpu/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/mike/anaconda2/envs/tfgpu/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/mike/anaconda2/envs/tfgpu/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/mike/anaconda2/envs/tfgpu/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/home/mike/anaconda2/e

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [16]:
# Neural implementation
def testNN(c):
    t = datetime.now()
    num_labels = 10
    pixelCount = 28 * 28
    train_subset = 500
    num_steps = 2001
    hiddenNodes = 1024
    useGPU = 1
    
    graph = tf.Graph()
    with graph.as_default():
        #with tf.device('/gpu:1'):
            # Configure Constants
        tf_TrainSet = tf.constant(train_dataset[:train_subset, :])
        tf_TrainLabels = tf.constant(train_labels[:train_subset, :])
        tf_TestSet = tf.constant(test_dataset)
        tf_ValidSet = tf.constant(valid_dataset)

        # Setup the logistic regression variables
        w1 = tf.Variable(tf.truncated_normal([pixelCount, hiddenNodes]))
        b1 = tf.Variable(tf.zeros([hiddenNodes]))
        w2 = tf.Variable(tf.truncated_normal([hiddenNodes, num_labels]))
        b2 = tf.Variable(tf.zeros([num_labels]))
        #keepProbability = tf.placeholder(tf.float32)
        keepProbability = 0.5
        # Build the model
        logitLayer1 = tf.matmul(tf_TrainSet, w1) + b1
        reluLayer1 = tf.nn.relu(logitLayer1)
        dropout = tf.nn.dropout(reluLayer1, keepProbability)
        logitLayer2 = tf.matmul(dropout, w2) + b2
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logitLayer2, tf_TrainLabels))
        # L2 regularization for the fully connected parameters.
        regularizers = (tf.nn.l2_loss(w1) + tf.nn.l2_loss(b1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(b2))
        # Add the regularization term to the loss.
        loss += c * regularizers

        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

        train_prediction = tf.nn.softmax(logitLayer2)
        valid_prediction = tf.nn.softmax(
            tf.matmul(tf.nn.relu(tf.matmul(tf_ValidSet, w1) + b1), w2) + b2)
        test_prediction = tf.nn.softmax(
            tf.matmul(tf.nn.relu(tf.matmul(tf_TestSet, w1) + b1), w2) + b2)

    with tf.Session(graph=graph) as session:
        # This is a one-time operation which ensures the parameters get initialized as
        # we described in the graph: random weights for the matrix, zeros for the
        # biases. 
        tf.initialize_all_variables().run()
        print('Initialized for c={}'.format(c))
        for step in range(num_steps):
            # Run the computations. We tell .run() that we want to run the optimizer,
            # and get the loss value and the training predictions returned as numpy
            # arrays.
            _, l, predictions = session.run([optimizer, loss, train_prediction])
            if (step % 200 == 0):
                nt = (datetime.now() - t).total_seconds()
                ac = accuracy(test_prediction.eval(), test_labels)
                print("Progress at {} steps after {} seconds has test accuracy of {:.2%}%".format(step, nt, ac/100))
                sleep(5) # Cool down so my laptop GPU doesn't catch fire
            if (step % 1000 == 0 and step > 0):
                print('Loss at step %d: %f' % (step, l))
                print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
                print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
                print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
        print('')
        
vals = [0.0075]
for v in vals:
    testNN(v)
    sleep(60) # Cool down so my laptop GPU doesn't catch fire

### For a batch of only 100 samples over 1000 steps without dropout:
# Loss at step 1000: 1.397398
# Training accuracy: 100.0%
# Validation accuracy: 68.7%
# Test accuracy: 74.7%

Initialized for c=0.0075
Progress at 0 steps after 0.697028 seconds has test accuracy of 33.48%%
Progress at 200 steps after 8.384267 seconds has test accuracy of 84.77%%
Progress at 400 steps after 16.033744 seconds has test accuracy of 85.05%%
Progress at 600 steps after 23.668059 seconds has test accuracy of 85.41%%
Progress at 800 steps after 31.305748 seconds has test accuracy of 86.00%%
Progress at 1000 steps after 38.951398 seconds has test accuracy of 86.33%%
Loss at step 1000: 1.510048
Training accuracy: 100.0%
Validation accuracy: 79.5%
Test accuracy: 86.3%
Progress at 1200 steps after 46.753556 seconds has test accuracy of 86.24%%
Progress at 1400 steps after 54.394699 seconds has test accuracy of 86.09%%
Progress at 1600 steps after 62.287301 seconds has test accuracy of 86.12%%
Progress at 1800 steps after 69.951711 seconds has test accuracy of 86.03%%
Progress at 2000 steps after 77.591171 seconds has test accuracy of 85.98%%
Loss at step 2000: 0.229812
Training accuracy:

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [69]:
# My laptop smells like burning when the GPU hits 90C so I needed a way to throttle it. Adding 
# 10 seconds per 100 epochs just slowed things down a ton and still allowed it to creep over 
# eventually so this is a bit cleaner in terms of not wasting time when it's cool enough and 
# not just plowing ahead when it hasn't gone down enough.

import commands

def checkTemp(gpuID):
    everything=commands.getoutput("nvidia-smi -q -d temperature")
    rows = everything.split('\n')
    rows = [r.strip() for r in rows[7:] if 'Current' in r]
    temp = int(rows[gpuID].split(':')[1].split()[0])
    return(temp)


def waitUntilBelow(goalTemp, GPUID, tripped):
    ret = tripped
    currentTemp = checkTemp(GPUID)
    if currentTemp < 0:
        print('ERROR CHECKING TEMPERATURE, MANUAL HOLDING')
    elif currentTemp > goalTemp + 10:
        print('High - Sleeping to cool down from {}'.format(currentTemp))
        sleep(10)
        ret = waitUntilBelow(goalTemp, GPUID, tripped + 10)
    elif currentTemp > goalTemp:
        sleep(2)
        ret = waitUntilBelow(goalTemp, GPUID, tripped + 2)
    
    return(ret)

In [68]:
t = datetime.now()
num_labels = 10
pixelCount = 28 * 28
train_subset = 25000
num_steps = 2001
hiddenNodes = 1024
n = 0

graph = tf.Graph()
with graph.as_default():
    with tf.device('/gpu:1'):
        # Configure Constants
        tf_TrainSet = tf.constant(train_dataset[:train_subset, :])
        tf_TrainLabels = tf.constant(train_labels[:train_subset, :])
        tf_TestSet = tf.constant(test_dataset)
        tf_ValidSet = tf.constant(valid_dataset)

        # Setup the logistic regression variables
        w1 = tf.Variable(tf.truncated_normal([pixelCount, hiddenNodes]), name='w1')
        b1 = tf.Variable(tf.zeros([hiddenNodes]), name='b1')
        w2 = tf.Variable(tf.truncated_normal([hiddenNodes, num_labels]), name='w2')
        b2 = tf.Variable(tf.zeros([num_labels]), name='b2')
        #keepProbability = tf.placeholder(tf.float32)
        keepProbability = 0.5
        # Build the model
        logitLayer1 = tf.matmul(tf_TrainSet, w1) + b1
        reluLayer1 = tf.nn.relu(logitLayer1)
        dropout = tf.nn.dropout(reluLayer1, keepProbability)
        logitLayer2 = tf.matmul(dropout, w2) + b2
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logitLayer2, tf_TrainLabels))
        
        # L2 regularization for the fully connected parameters.
        regularizers = (tf.nn.l2_loss(w1) + tf.nn.l2_loss(b1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(b2))
        # Add the regularization term to the loss.
        c = 0.0075
        loss += c * regularizers

        global_step = tf.Variable(0, trainable=False, name='gStep')
        starter_learning_rate = 0.1
        learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                                   1000, 0.96, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

        train_prediction = tf.nn.softmax(logitLayer2)
        valid_prediction = tf.nn.softmax(
            tf.matmul(tf.nn.relu(tf.matmul(tf_ValidSet, w1) + b1), w2) + b2)
        test_prediction = tf.nn.softmax(
            tf.matmul(tf.nn.relu(tf.matmul(tf_TestSet, w1) + b1), w2) + b2)
    config = tf.ConfigProto(allow_soft_placement = True)
    with tf.Session(graph=graph, config = config) as session:
        # This is a one-time operation which ensures the parameters get initialized as
        # we described in the graph: random weights for the matrix, zeros for the
        # biases. 
        tf.initialize_all_variables().run()
        print('Initialized for c={}'.format(c))
        for step in range(num_steps):
            # Run the computations. We tell .run() that we want to run the optimizer,
            # and get the loss value and the training predictions returned as numpy
            # arrays.
            _, l, predictions = session.run([optimizer, loss, train_prediction])
            if (step % 10 == 0 and step > 0):
                n += waitUntilBelow(70, 1, 0)
                # Cool down so my laptop GPU doesn't catch fire
            if (step % 200 == 0 and step > 0):
                nt = (datetime.now() - t).total_seconds()
                ac = accuracy(test_prediction.eval(), test_labels)
                print('There were {} cooling cycles this round.'.format(n))
                n = 0
                print("Progress at {} steps after {} seconds has test accuracy of {:.2%}%".format(step, nt, ac/100))
            if (step % 1000 == 0 and step > 0):
                print('Loss at step %d: %f' % (step, l))
                print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
                print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
                print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
        print('')

Initialized for c=0.0075
There were 36 cooling cycles this round.
Progress at 200 steps after 138.988119 seconds has test accuracy of 89.92%%
There were 60 cooling cycles this round.
Progress at 400 steps after 301.279163 seconds has test accuracy of 90.61%%
There were 68 cooling cycles this round.
Progress at 600 steps after 471.632724 seconds has test accuracy of 90.80%%
There were 68 cooling cycles this round.
Progress at 800 steps after 641.993727 seconds has test accuracy of 90.73%%
There were 70 cooling cycles this round.
Progress at 1000 steps after 814.405416 seconds has test accuracy of 90.77%%
Loss at step 1000: 518.164978
Training accuracy: 82.0%
Validation accuracy: 84.3%
Test accuracy: 90.8%
There were 70 cooling cycles this round.
Progress at 1200 steps after 986.869072 seconds has test accuracy of 90.86%%
There were 70 cooling cycles this round.
Progress at 1400 steps after 1159.281995 seconds has test accuracy of 91.03%%
There were 68 cooling cycles this round.
Progress